In [ ]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    print(lines[-2:])
    print(str_lines[-2:])
    return str_lines

elf_data = readin("Day14input.txt")

# --- Day 14: Regolith Reservoir ---

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads *behind* the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. 

*https://adventofcode.com/2018/day/17*

You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly *air* with structures made of *rock*.


Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:

    498,4 -> 498,6 -> 496,6
    503,4 -> 502,4 -> 502,9 -> 494,9


In [ ]:
test_data = ["498,4 -> 498,6 -> 496,6",
"503,4 -> 502,4 -> 502,9 -> 494,9"]

In [ ]:
def parse_data(data):
    pd = []
    for d in data:
        path = []
        temp = d.split()
        for i in range(0, len(temp), 2):
            temp2 = temp[i].split(",")
            path.append((int(temp2[0]), int(temp2[1])))
        pd.append(path)
    return pd

rock_paths_test = parse_data(test_data)
rock_paths_elf = parse_data(elf_data)


This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.


In [ ]:
entry_point_plus = (500, 0)

# x represents distance to the right and y represents distance down. 
# so a slice[x][y] is 

#  0,0   1,0   2,0   3,0
#  0,1   1,1   2,1   3,1
#  0,2   1,2   2,2   3,2 
#  0,3   1,3   2,3   3,3 

def init_slice(min_x, max_x, num_y, entry):
    slice=[]
    for y in range(0,num_y):
        new_line = []
        for x in range(min_x, max_x+1):    
            if (x,y) == entry:
                new_line.append("+")
            else:
                new_line.append(".")
        slice.append(new_line)
    return slice

def printout(st):
    for s in st:
        print(s)

def get_grid_dim(pd):
    min_x = pd[0][0][0]
    max_x = pd[0][0][0]
    num_y = pd[0][0][1]
    for i in pd:
        for j in i:
            # print(i)
            # print(j)
            # print(j[0])
            # print(j[1])
            num_y = max(j[1], num_y)
            min_x = min(j[0], min_x)
            max_x = max(j[0], max_x)

    return (min_x, max_x, (num_y+1))

# (min_x, max_x, num_y) = get_grid_dim(rock_paths_test)
# print(min_x, max_x, num_y)

# slice_test = init_slice(min_x, max_x, num_y, entry_point_plus)
# printout(slice_test)

In [ ]:
def lookup(data, min_x, x, y):
    #print("LOOKUP ", min_x, x, y, len(data[0]), len(data))
    return data[y][(x-min_x)]

#print(lookup(slice_test, min_x, 500, 0))

def edit(data, min_x, x, y, new):
    data[y][(x-min_x)] = new
    return data

In [ ]:
def process_paths(paths, slice_data, min_x):
    for p in paths:
        # print(p)
        for i in range(0, len(p)-1):
            start = p[i]
            end = p[i+1]
            # print(start,end)
            if start[0] > end[0]:
                x_step = -1
            else: 
                x_step = 1
            for x in range(start[0], end[0]+x_step, x_step):
                if start[1] > end[1]:
                    y_step = -1
                else: 
                    y_step = 1
                for y in range(start[1], end[1]+y_step, y_step):
                   # print(min_x, x, y, "#", len(slice_data), len(slice_data[0]))
                    slice_data = edit(slice_data, min_x, x, y, "#")
    return slice_data

# slice_test_w_paths = process_paths(rock_paths_test, slice_test, min_x)
# printout(slice_test_w_paths)




Drawing rock as #, air as ., and the source of the sand as +, this becomes:


    4     5  5
    9     0  0
    4     0  3
    0 ......+...
    1 ..........
    2 ..........
    3 ..........
    4 ....#...##
    5 ....#...#.
    6 ..###...#.
    7 ........#.
    8 ........#.
    9 #########.



Sand is produced *one unit at a time*, and the next unit of sand is not produced until the previous unit of sand *comes to rest*. A unit of sand is large enough to fill one tile of air in your scan.

* A unit of sand always falls *down one step* if possible. 

* If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally *one step down and to the left*. 

* If that tile is blocked, the unit of sand attempts to instead move diagonally *one step down and to the right*. 

* Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. 

* If all three possible destinations are blocked, the unit of sand *comes to rest* and no longer moves, at which point the next unit of sand is created back at the source.


In [ ]:
def new_co_ords(xy, dir): 
    if dir=="down":
        return (xy[0], (xy[1]+1))
    elif dir=="downleft":
        return ((xy[0]-1), (xy[1]+1))
    elif dir=="downright": 
        return ((xy[0]+1), (xy[1]+1))
    else: 
        return None



In [ ]:
def expand_grid(data, min_x, max_x, amount):
    new_min_x = min_x - amount
    new_max_x = max_x + amount
    old_first = data[0] #just for debugging
    pre = []
    post = []
    for min in range(new_min_x, min_x):
        pre.append(".")
    for max in range(max_x, new_max_x):
        post.append(".")
    for d in range(0, len(data)):
        data[d] = pre+data[d]+post
    #check this is right
    if (new_max_x - new_min_x +1) != len(data[0]):
        print("issue in expand_grid ", new_min_x, new_max_x, len(data), data[0], pre, old_first, post, min_x, max_x, amount)
    return (data, new_min_x, new_max_x)

def new_co_ords_in_grid(data, new_x, new_y, min_x, max_x, final_status_known, time_to_stop, part):
    out_of_range = new_x < min_x or new_x > max_x
    if out_of_range is True:
        if part ==1:
            print ("ABYSS")
            final_status_known = True # just to escape out
            time_to_stop = True
        else:
            # going to have to expand the grid 
            amount = 100 * max(new_x-max_x, min_x-new_x)
            (data, min_x, max_x) = expand_grid(data, min_x, max_x, amount)
    return ((not out_of_range), final_status_known, time_to_stop)

def drop_sand(data, min_x, max_x, time_to_stop, part):
    current_s = entry_point_plus
    final_status_known = False
    moves = ["down", "downleft", "downright"]
    i = 0   
    while not final_status_known:
        (new_x, new_y) = new_co_ords(current_s, moves[i])
        (check, final_status_known, time_to_stop) = new_co_ords_in_grid(data, new_x, new_y, min_x, max_x, final_status_known, time_to_stop, part)
        if check:
            if lookup(data, min_x, new_x, new_y) == ".":
                current_s = (new_x, new_y)
                i = 0
            else:
                if i < 2:
                    i = i+1
                else:
                    final_status_known = True
                    data = edit(data, min_x, current_s[0], current_s[1], "o")
                    if current_s[0] == 500 and current_s[1] == 0:   #hardcoded, lazy, I know :)
                        print("blocking start")
                        time_to_stop = True
                    i = 0
    # printout(data)
    # print()
    return data, time_to_stop

#print(how_much_sand(entry_point_plus, rock_paths_test, 1))  


In [ ]:
def how_long_till_stopped(data, min_x, max_x, part):
    s = 0
    time_to_stop = False
    while not time_to_stop:
        (data, time_to_stop) = drop_sand(data, min_x, max_x, time_to_stop, part)
        if not time_to_stop:
            s = s+1
    if part==2: 
        return s+1  #hack
    else:
        return s


In [ ]:
def how_much_sand(entry, rock_paths_data, part):
    (min_x, max_x, num_y) = get_grid_dim(rock_paths_data)
    if part==2:
        num_y = num_y+2
        min_x = min_x - 20 # for now. will need to make dynamic. Oh irony, Stage 2 students will be happy
        max_x = max_x + 20 # for now. will need to make dynamic. Oh irony, Stage 2 students will be happy
        rock_paths_data.append([(min_x, num_y-1), (max_x, num_y-1)])

    slice_data = init_slice(min_x, max_x, num_y, entry_point_plus)
    slice_data_w_paths = process_paths(rock_paths_data, slice_data, min_x)
    #printout(slice_data_w_paths)
    
    s = how_long_till_stopped(slice_data_w_paths, min_x, max_x, part)
    return s



In [76]:
#print(how_much_sand(entry_point_plus, rock_paths_test, 2))  
#print(how_much_sand(entry_point_plus, rock_paths_test, 2))        
#print(how_much_sand(entry_point_plus, rock_paths_elf, 1))
print(how_much_sand(entry_point_plus, rock_paths_elf, 2))


So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:


    ......+...
    ..........
    ..........
    ..........
    ....#...##
    ....#...#.
    ..###...#.
    ........#.
    ......o.#.
    #########.

The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:


    ......+...
    ..........
    ..........
    ..........
    ....#...##
    ....#...#.
    ..###...#.
    ........#.
    .....oo.#.
    #########.
    
After a total of five units of sand have come to rest, they form this pattern:


    ......+...
    ..........
    ..........
    ..........
    ....#...##
    ....#...#.
    ..###...#.
    ......o.#.
    ....oooo#.
    #########.

After a total of 22 units of sand:


    ......+...
    ..........
    ......o...
    .....ooo..
    ....#ooo##
    ....#ooo#.
    ..###ooo#.
    ....oooo#.
    ...ooooo#.
    #########.

Finally, only two more units of sand can possibly come to rest:


    ......+...
    ..........
    ......o...
    .....ooo..
    ....#ooo##
    ...o#ooo#.
    ..###ooo#.
    ....oooo#.
    .o.ooooo#.
    #########.

Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:


    .......+...
    .......~...
    ......~o...
    .....~ooo..
    ....~#ooo##
    ...~o#ooo#.
    ..~###ooo#.
    ..~..oooo#.
    .~o.ooooo#.
    ~#########.
    ~..........
    ~..........
    ~..........
    
Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?



# --- Part Two ---

You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a y coordinate equal to two plus the highest y coordinate of any point in your scan.


In [ ]:
#print(how_much_sand(entry_point_plus, rock_paths_test, 2))        
#print(how_much_sand(entry_point_plus, rock_paths_elf, 2))


In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. (This is as if your scan contained one extra rock path like -infinity,11 -> infinity,11.) With the added floor, the example above now looks like this:

            ...........+........
            ....................
            ....................
            ....................
            .........#...##.....
            .........#...#......
            .......###...#......
            .............#......
            .............#......
            .....#########......
            ....................
    <-- etc #################### etc -->

To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at 500,0, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after 93 units of sand come to rest:

    ............o............
    ...........ooo...........
    ..........ooooo..........
    .........ooooooo.........
    ........oo#ooo##o........
    .......ooo#ooo#ooo.......
    ......oo###ooo#oooo......
    .....oooo.oooo#ooooo.....
    ....oooooooooo#oooooo....
    ...ooo#########ooooooo...
    ..ooooo.......ooooooooo..
    #########################

Using your scan, simulate the falling sand until the source of the sand becomes blocked. How many units of sand come to rest?